<a href="https://colab.research.google.com/github/fionatjahjono/ml_project/blob/main/recommender_system_content_based_filtering_ps133.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import gensim.downloader
from sklearn.model_selection import train_test_split

In [ ]:
app = Flask(__name__)

@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.get_json(force=True)
    user_id_to_check = data['user_id']
    threshold = data['threshold']

In [ ]:
    df_user = pd.read_csv("/content/user_table.csv", index_col=0)
    df_user = df_user.drop("user_name", axis=1)


    df_user

In [ ]:
    df_user[['user_interest', 'user_past_event']] = df_user[['user_interest', 'user_past_event']].apply(lambda x: x.str.split(', '))
    df_user_expanded = df_user.explode('user_interest').explode('user_past_event')
    df_user_expanded['row_number'] = range(1, len(df_user_expanded) + 1)

    df_user_expanded


In [ ]:
    df_event = pd.read_csv("/content/event_table.csv", index_col=0)
    df_event = df_event.drop(["index_to_drop", "date", "event_price", "split"], axis=1)

    df_event

In [ ]:
    train, test = train_test_split(df_user, test_size=0.2, random_state=42)

    train['user_id'] = train.index.astype(int)
    test['user_id'] = test.index.astype(int)

In [ ]:
    print("Number of user in train:",train.shape[0])
    print("Number of user in test:",test.shape[0])

In [ ]:
    df_event["combined"] = df_event["event_name"] + " " + df_event["event_description"]
    df_event.combined

In [ ]:
    nltk.download("punkt")
    nltk.download("stopwords")

In [ ]:
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize

In [ ]:
    df_event["combined"] = df_event["combined"].str.lower()
    df_event.combined

In [ ]:
    def tokenize_text(x):
        if pd.isna(x):
            return []
        x = re.sub(r'\d+', '', x)
        return word_tokenize(x.lower())

    df_event["tokenized"] = df_event["combined"].apply(tokenize_text)

In [ ]:
df_event["tokenized"]

In [ ]:
    df_event["tokenized"] = df_event["combined"].apply(tokenize_text)

    df_event["clean_tokenized"] = df_event["tokenized"].apply(lambda tokens: [word for word in tokens if word.isalpha() and word not in stopwords.words("english")])
    df_event.drop(columns=["combined", "tokenized"], inplace=True)

In [ ]:
    glove = gensim.downloader.load('glove-wiki-gigaword-100')

In [ ]:
    def get_embedding(list_of_tokens):
        embeddings = np.zeros(100)
        for token in list_of_tokens:
            if token in glove:
                embeddings += glove[token]
        return embeddings

In [ ]:
    df_event["embedding"] = df_event["clean_tokenized"].apply(lambda x: get_embedding(x))

In [ ]:
    df_event["embedding"]

In [ ]:
    # for user_id = 1
    u = 1
    user_data = train[train["user_id"] == u]
    # Combine user interests and past events
    items_of_user_1 = user_data["user_interest"].explode().astype(int).values
    past_events_of_user_1 = user_data["user_past_event"].explode().astype(int).values
    combined_items_of_user_1 = np.concatenate((items_of_user_1, past_events_of_user_1))

    embedding_of_events_of_user = df_event.loc[df_event.category_id.isin(items_of_user_1), "embedding"]
    profile_user = np.sum(embedding_of_events_of_user.values)

    profile_user = np.sum(embedding_of_events_of_user.values)
    profile_user

    profile_user.shape

    df_event["cosine"] = df_event.embedding.apply(lambda x: cosine_similarity(profile_user.reshape(1, 100), x.reshape(1, 100))).apply(lambda x: x[0][0])
    df_event_top50 = df_event.sort_values("cosine", ascending=False).head(50)
    top50item = df_event_top50.index.values

    top50item = df_event.reset_index().sort_values("cosine", ascending=False).head(50).iloc[:, 0].values
    top50item

    print(test.dtypes)

In [ ]:
    # Display top 50 recommended items for the user in the test set
    df_event_top50.head(10)

    if user_id_to_check not in test["user_id"].values:
        return jsonify({"error": f"User {user_id_to_check} does not exist in the test data."})
    else:
        relevant_items_above_threshold = df_event_top50[df_event_top50["cosine"] > threshold].index.values
        return jsonify({"Recommend Event": relevant_items_above_threshold.tolist()})

if __name__ == '__main__':
    app.run(debug=True)